`[БРД051.2] Предсказание вероятности отказа насоса`

Для решения задачи прогнозирования времени работы насосов по их характеристикам можно использовать не только классические методы машинного обучения, но и аппарат анализа выживаемости произвольных объектов, реализованный в библиотеках `lifelines` и `scikit-survival` для языка программирования `Python`.

Данные модели и алгоритмы строят вероятностные модели, позволяющие получать распределение вероятностей выхода технологического оборудования (с заданными технологическими параметрами) из строя в определенный день.

Данную задачу можно решать несколькими подходами:
1. использовать полупараметрические модели (модели, которые на основании второстепенных признаков строят прогнозы, в том числе с учетом образа вида распределения целевого признака);
2. использовать параметрические модели (модели, которые полностью берут за основу вид распределения параметра, отвечающего за продолжительность работы устройства, и на его основе строят прогноз);
3. использовать модели, основанные на алгоритмах машинного обучения.

При этом эмпирические модели (Каплан-Мейер, Нельсон-Аален и модель кумулятивной опасности) не являются прогнозными, поскольку они исключительно позволяют визуализировать текущие исторические наблюдения и строить по ним аналитику.

Для того, чтобы решить поставленную задачу, необходимо обработать данные, а после с их помощью построить соответствующие модели и оценить каждую из них, чтобы отобрать наилучшую -- ту, которая строит вероятностный прогноз наиболее правдоподобно.

`Содержание ноутбука`

**Table of contents**<a id='toc0_'></a>    
- 1. [Раздел 1. Обработка входных данных и датасета](#toc1_)    
  - 1.1. [Research](#toc1_1_)    
    - 1.1.1. [Загрузка исходных данных](#toc1_1_1_)    
    - 1.1.2. [Анализ типов данных и значений признаков](#toc1_1_2_)    
      - 1.1.2.1. [Выделение фрагментов исходного датасета](#toc1_1_2_1_)    
      - 1.1.2.2. [Анализ выделенных фрагментов](#toc1_1_2_2_)    
    - 1.1.3. [Формирование обучающей и тестирующей выборок](#toc1_1_3_)    
    - 1.1.4. [Обработка категориальных признаков](#toc1_1_4_)    
      - 1.1.4.1. [Выделение фрагментов категориальных данных](#toc1_1_4_1_)    
      - 1.1.4.2. [Кодирование категориальных данных](#toc1_1_4_2_)    
        - 1.1.4.2.1. [Представление категорий, в которых есть отношение порядка](#toc1_1_4_2_1_)    
        - 1.1.4.2.2. [Представление категорий сложной структуры](#toc1_1_4_2_2_)    
        - 1.1.4.2.3. [Представление равнозначных категорий](#toc1_1_4_2_3_)    
      - 1.1.4.3. [Формирование категориального датасета](#toc1_1_4_3_)    
    - 1.1.5. [Обработка вещественных признаков](#toc1_1_5_)    
      - 1.1.5.1. [Выделение вещественного датасета](#toc1_1_5_1_)    
      - 1.1.5.2. [Заполнение пропущенных значений в датасете](#toc1_1_5_2_)    
        - 1.1.5.2.1. [Первичный анализ пропущенных значений](#toc1_1_5_2_1_)    
        - 1.1.5.2.2. [Визуализация пропущенных значений](#toc1_1_5_2_2_)    
        - 1.1.5.2.3. [Заполнение пропущенных значений](#toc1_1_5_2_3_)    
    - 1.1.6. [Формирование предобработанного датасета](#toc1_1_6_)    
  - 1.2. [Production](#toc1_2_)    
  - 1.3. [`Выводы по разделу`](#toc1_3_)    
- 2. [Раздел 2. Генерация признаков](#toc2_)    
  - 2.1. [Research](#toc2_1_)    
    - 2.1.1. [Генерация сводной статистической таблицы численного датасета](#toc2_1_1_)    
    - 2.1.2. [Применение скалирования к вещественным признакам](#toc2_1_2_)    
      - 2.1.2.1. [Применение стандартизации к вещественным признакам](#toc2_1_2_1_)    
      - 2.1.2.2. [Применение MinMax-преобразования к вещественным признакам](#toc2_1_2_2_)    
    - 2.1.3. [Проверка наличия мультиколлинеарности в данных](#toc2_1_3_)    
    - 2.1.4. [Генерация нового признакового пространства](#toc2_1_4_)    
      - 2.1.4.1. [Понижение пространства для исходных данных](#toc2_1_4_1_)    
      - 2.1.4.2. [Понижение пространства для стандартизированных данных](#toc2_1_4_2_)    
      - 2.1.4.3. [Понижение пространства для данных после MinMax-преобразования](#toc2_1_4_3_)    
    - 2.1.5. [Отбор признаков при помощи устранения мультиколлинеарности](#toc2_1_5_)    
  - 2.2. [Production](#toc2_2_)    
  - 2.3. [`Выводы по разделу`](#toc2_3_)    
- 3. [Раздел 3. Обучение ML-моделей](#toc3_)    
  - 3.1. [Research](#toc3_1_)    
    - 3.1.1. [Построение параметрических моделей](#toc3_1_1_)    
      - 3.1.1.1. [Формирование вариационного ряда](#toc3_1_1_1_)    
      - 3.1.1.2. [Проверка ряда на экспоненциальное распределение](#toc3_1_1_2_)    
      - 3.1.1.3. [Проверка ряда на распределение Вейбулла](#toc3_1_1_3_)    
  - 3.2. [Production](#toc3_2_)    
    - 3.2.1. [Модель пропорциональной опасности Кокса](#toc3_2_1_)    
    - 3.2.2. [Аддитивная модель Аалена](#toc3_2_2_)    
  - 3.3. [`Выводы по разделу`](#toc3_3_)    
- 4. [Раздел 4. Кросс-валидация и подбор гиперпараметров](#toc4_)    
  - 4.1. [Production](#toc4_1_)    
    - 4.1.1. [Обучение моделей на данных после PCA](#toc4_1_1_)    
      - 4.1.1.1. [Обучение модели Кокса](#toc4_1_1_1_)    
        - 4.1.1.1.1. [Обучение на классическом датасете](#toc4_1_1_1_1_)    
        - 4.1.1.1.2. [Обучение на стандартизированном датасете](#toc4_1_1_1_2_)    
        - 4.1.1.1.3. [Обучение на датасете с MinMax-преобразованием](#toc4_1_1_1_3_)    
      - 4.1.1.2. [Обучение аддитивной модели Аалена](#toc4_1_1_2_)    
        - 4.1.1.2.1. [Обучение на классическом датасете](#toc4_1_1_2_1_)    
        - 4.1.1.2.2. [Обучение на стандартизированном датасете](#toc4_1_1_2_2_)    
        - 4.1.1.2.3. [Обучение на датасете с MinMax-преобразованием](#toc4_1_1_2_3_)    
    - 4.1.2. [Обучение моделей на данных после ручного отбора признаков](#toc4_1_2_)    
      - 4.1.2.1. [Обучение модели Кокса](#toc4_1_2_1_)    
        - 4.1.2.1.1. [Обучение на классическом датасете](#toc4_1_2_1_1_)    
        - 4.1.2.1.2. [Обучение на стандартизированном датасете](#toc4_1_2_1_2_)    
        - 4.1.2.1.3. [Обучение на датасете с MinMax-преобразованием](#toc4_1_2_1_3_)    
      - 4.1.2.2. [Обучение аддитивной модели Аалена](#toc4_1_2_2_)    
        - 4.1.2.2.1. [Обучение на классическом датасете](#toc4_1_2_2_1_)    
        - 4.1.2.2.2. [Обучение на стандартизированном датасете](#toc4_1_2_2_2_)    
        - 4.1.2.2.3. [Обучение на датасете с MinMax-преобразованием](#toc4_1_2_2_3_)    
  - 4.2. [`Выводы по разделу`](#toc4_2_)    
- 5. [Раздел 5. Предсказание на новых данных](#toc5_)    
  - 5.1. [Production](#toc5_1_)    
  - 5.2. [`Выводы по разделу`](#toc5_2_)    
- 6. [`Выводы по исследованию`](#toc6_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

`Структура ноутбука`

Каждый раздел состоит из двух частей:
 - `research (необязательная)` — включает в себя вспомогательные исследования, тесты, визуализации, диаграммы и аналитические выкладки, используемые в процессе работы;
 - `production (обязательная)` — содержит финальный, чистый и воспроизводимый код, который используется для запуска и проверки.

При этом:
1. research-часть не обязательна и может отсутствовать в некоторых разделах;
2. production-часть не содержит закомментированный код и ветки решений, которые не используются в ходе решения;
3. код в Production-части должен быть оформлен с принципами воспроизводимости и чистоты, избежания лишних промежуточных вычислений и отладочных выводов, и **он будет использоваться для переноса в PY-файлы в будущем**.


`Работа в рамках устранения замечаний по валидации`

Данная работа проходит повторную валидацию. В отчете были выявлены следующие проблемы:

| TODO                                                                                                                                   | Приоритет        | Комментарий                                                                                                                                                                                                                                                                                                                                             |
| -------------------------------------------------------------------------------------------------------------------------------------- | ---------------- | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| В столбце "Мероприятия остановок" потенциально можно объеденить разные названия                                                        | Важность низкая  | Для примера в списке уникальных мероприятий 32 с типом содержащем "Исследования"                                                                                                                                                                                                                                                                        |
| Лучше для test отобрать чуть больше значений (хотя бы 5% от выборки)                                                                   | Важность низкая  |                                                                                                                                                                                                                                                                                                                                                         |
| По описанию "Признак отказа - Булевый признак, указывающий, был ли насос выведен из работы из-за поломки"                              | Важность высокая | Точно ли living_id имеет Признак отказа = 1, а death_id имеет Признак отказа = 0, а не наоборот?                                                                                                                                                                                                                                                        |
| При использовании медианы качество будет лучше                                                                                         | Важность средняя | Лучше выводить графики (до того как заполнять и смотреть распределения) и смотреть глазами                                                                                                                                                                                                                                                              |
| При использовании скользящего окна (усредненее 2 значений до, 2 значений после искомого) может быть еще лучше                          | Важность средняя |                                                                                                                                                                                                                                                                                                                                                         |
| (Для оптимизации) для заполнения пропусков можно использовать MICE, он позволяет перебирать модели для заполнения                      | Важность низкая  |                                                                                                                                                                                                                                                                                                                                                         |
| Надо сделать финальную проверку модели Кокса                                                                                           | Важность высокая | Проверка пропорциональности рисков + Анализа остатков модели                                                                                                                                                                                                                                                                                            |
| Из отчета ниже видно, что модель имеет нарушения предположения пропорциональности рисков для нескольких переменных                     | Важность высокая | Влияние этих переменных на риск изменяется со временем, что требует корректировки модели<br>Например, для категориальных переменных (Группа по кол-ву циклов ПКВ, year) можно использовать strata в fit()<br>Для непрерывных переменных (Pзаб, Минуты вращения насоса) можно использовать логарифмические преобразования (необходимо подробнее изучить) |
| Также из отчета ниже proportional_hazard_test можное увидеть несоблюдение пропорциональности рисков (по тем же переменным, что и выше) | Важность высокая |                                                                                                                                                                                                                                                                                                                                                         |
| Также необходимо провести анализа остатков модели                                                                                      | Важность высокая |                                                                                                                                                                                                                                                                                                                                                         |

В результате мероприятий по устранению замечаний были выполнены следующие действия:
1. с заказчиком был проработан вопрос семантики признаков для того, чтобы установить, требуется ли кодировать "Мероприятия остановок";
2. был собран новый датасет, который включает как объекты, вышедшие из строя, так и те, что еще работают по сей день, с целью увеличения обучающей выборки и формирования тестовой для проверки метрик качества;
3. была исправлена ошибка, в результате которой значения "Признака отказа" были перепутаны;
4. при заполнении пропусков была исследована медиана при помощи анализа распределений и KL-дивергенции;
5. приведена аргументация, почему применение методов скользящего окна и алгоритма MICE для заполнения пропусков не подходит для данной задачи;
6. была выполнена итоговая проверка модели Кокса, в результате которой было принято решение использовать методы машинного обучения для построения моделей выживаемости, поскольку важные статистические свойства, такие как пропорциональность рисков, не выполняются на исходном датасете, а метрики качества на преобразованных данных сильно хуже.

`Формирование окружения для работы`

Первоначально необходимо подготовить окружение для работы с данными и их моделирования. Чтобы не возникало конфликтов виртуальной среды Python, зафиксируем используемые в ходе исследования версии основных библиотек и фреймворков, которые будут использоваться далее. А также пропишем последовательность команд для формирования среды.

Работа справедлива для версии `Python3.9+` (выполнялась на платформе N1).

Первоначально установим необходимые библиотеки при помощи пакетного менеджера `pip` в виртуальное окружение.

Также зафиксируем версии библиотек далее.

// ячейка для версий библиотек
// сами версии можно получить с помощью команды: 

```
pip list --format=freeze
```

Данные библиотеки можно установить, поместив их вместе с версиями в файл `requirements.txt`. И выполнимв команду ниже.

In [ ]:
!pip install --upgrade --force-reinstall -r requirements.txt

Средствами Python импортируем необходимые для работы библиотеки. В силу традиции, присвоим некоторым модулям псевдонимы (например, pd для pandas и т.д.). Роль импортируемых библиотек и фреймворков представлена в таблице. 

| Наименование библиотеки | Описание                                                                             |
| ----------------------- | ------------------------------------------------------------------------------------ |
| matplotlib              | Библиотека для визуализации данных                                                   |
| pandas                  | Библиотека для работы таблицами и плоскими данными                                   |
| numpy                   | Библиотека для работы с массивами и матрицами                                        |
| scikit-learn            | Библиотека для построения классических моделей машинного обучения и работы с данными |
| scipy                   | Библиотека для работы со статистическими объектами                                   |
| missingno               | Библиотека для визуализации пропусков в данных                                       |
| seaborn                 | Библиотека для визуализации данных                                                   |
| lifelines               | Библиотека для обучения и построения статистических моделей выживаемости             |
| category_encoders       | Библиотека для кодирования категориальных признаков в вещественные значения          |
| typing                  | Библиотека для типизации парамеров функций и переменных                              |
| warnings                | Библиотека для корректировки выводов предупреждений                                  |
| scikit-survival         | Библиотека для построения моделей выживаемости, основанных на методах ML             |


In [ ]:
import pandas as pd
import numpy as np
import typing as t

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
import warnings
warnings.filterwarnings('ignore')

Для воспроизводимости результатов исследования зафиксируем основание генератора случайных чисел, который используется для деления выборки на обучение и тест, а также при обучении моделей.

In [5]:
seed = 5
np.random.seed(seed)

# 1. <a id='toc1_'></a>[Раздел 1. Обработка входных данных и датасета](#toc0_)

Исходный набор данных представляет из себя набор технологических параметров добывающего оборудования, полученный путем агрегации исторических наблюдений, в котором необходимо предсказать  распределение вероятностей выхода из строя по дням для данного объекта.

| **№** | **Признак** | **Описание** | **Тип данных признака** |
|---|---|---|---|
| **1** | Скважина | Уникальный номер скважины | Категориальный |
| **2** | Сборка | Наименование технологической сборки | Категориальный |
| **3** | Эра.Дата монтажа | Дата (ДД.ММ.ГГГГ), когда был произведен монтаж насоса | Агрегат данных (дата) |
| **4** | Дней в работе | Количествово дней, сколько насос проработал до демонтажа | Целочисленный |
| **5** | Кол-во циклов ПКВ | Количество циклов ПКВ, которые были осущетслены за время работы оборудования | Целочисленный |
| **6** | Кол-во циклов ПКВ/Дней в работе | Среднее количество циклов ПКВ в 1 день | Вещественный |
| **7** | sborka_id | Уникальный номер технологической сборки | Категориальный |
| **8** | well_id | Сцепка идентификатора скважины и даты | Категориальный |
| **9** | Причины остановок новые | Мероприятие остановки по УСОИ за время работы насоса | Категориальный |
| **10** | Bн [м3/м3] ТР | Удельный расход электроэнергии на одну тонну добытой жидкости | Вещественный |
| **11** | Признак отказа | Булевый признак, указывающий, был ли насос выведен из работы из-за поломки | Логический (0 или 1) |
| **12** | Режим гр-ка | Режимы работы скважины за время работы насоса | Категориальный |
| **13** | Группа | Наиболее частый режим работы скважины за время работы насоса | Категориальный |
| **14** | Группа ПКВ | Наиболее частый режим работы скважины ПКВ | Категориальный |
| **15** | УРЭ [кВтч/т] | Удельный расход электроэнергии на одну тонну добытой жидкости | Вещественный |
| **16** | УРЭ К [кВтч/т] | Удельный расход электроэнергии на одну тонну добытой жидкости с контроллера станции управления | Вещественный |
| **17** | Э/э (Wакт) [кВт*ч] | Электроэнергия расчетная от максимальной за сутки активной мощности | Вещественный |
| **18** | Э/э реакт. (ТМ) | Реактивная электроэнергия со счетчика станции управления | Вещественный |
| **19** | Э/э счет. (ТМ) [Вт] | Активная электроэнергия со счетчика станции управления | Вещественный |
| **21** | Номинальный дебит сборки [м3/сут] | Номинальный дебит сборки за время работы | Вещественный |
| **22** | Номинальный напор сборки [м] | Номинальный напор сборки за время работы | Вещественный |
| **23** | Номинальная мощность сборки [кВт] | Номинальная мощность сборки за время работы | Вещественный |
| **24** | Тип ствола | Код типа ствола (справочник OIS) | Категориальный |
| **25** | Qж OIS протяжка медиана [м3/сут] | Медианный дебит жидкости, подтвержденный в ОИС и протянутый на каждый день в работе | Вещественный |
| **26** | Qж OIS протяжка среднее [м3/сут] | Средний дебит жидкости, подтвержденный в ОИС и протянутый на каждый день в работе | Вещественный |
| **27** | Qж OIS протяжка ст. откл-е [м3/сут] | Среднеквадратическое отклонение дебита жидкости, подтвержденное в ОИС и протянутое на каждый день в работе | Вещественный |
| **28** | Qн OIS протяжка медиана [тн/сут] | Медианный дебит жидкости, подтвержденный в ОИС и протянутый на каждый день в работе | Вещественный |
| **29** | Qн OIS протяжка среднее [тн/сут] | Средний дебит жидкости, подтвержденный в ОИС и протянутый на каждый день в работе | Вещественный |
| **30** | Qн OIS протяжка ст. откл-е [тн/сут] | Среднеквадратическое отклонение дебита жидкости, подтвержденное в ОИС и протянутое на каждый день в работе | Вещественный |
| **31** | Обв-ть V OIS протяжка медиана [%] | Медианная обводненность, подтвержденная в ОИС и протянутая на каждый день в работе | Вещественный |
| **32** | Обв-ть V OIS протяжка среднее [%] | Средняя обводненность, подтвержденная в ОИС и протянутая на каждый день в работе | Вещественный |
| **33** | Обв-ть V OIS протяжка ст. откл-е [%] | Среднеквадратическое отклонение обводненности, подтвержденное в ОИС и протянутое на каждый день в работе | Вещественный |
| **34** | Pзаб медиана [ат] | Медианное забойное давление | Вещественный |
| **35** | Pзаб среднее [ат] | Среднее забойное давление | Вещественный |
| **36** | Pзаб ст. откл-е [ат] | Среднеквадратическое отклонение забойного давления | Вещественный |
| **37** | Pпл медиана [ат] | Медианное значение пластового давления | Вещественный |
| **38** | Pпласт среднее [ат] | Среднее значение пластового давления | Вещественный |
| **39** | Pпласт ст. откл-е [ат] | Среднеквадратическое отклонение пластового давления | Вещественный |
| **40** | Глубина насоса [м] | Глубина насоса в матрах | Вещественный |
| **41** | Дата начала экспл-ии | Дата (ДД.ММ.ГГГГ) начала эксплуатации насоса | Агрегат данных (дата) |
| **42** | Кол-во пластов | Количество эксплуатируемых пластов | Целочисленный |
| **43** | Hдин медиана [м] | Динамический уровень жидкости в скважине медиана | Вещественный |
| **44** | Hдин среднее [м] | Динамический уровень жидкости в скважине среднее | Вещественный |
| **45** | Hдин ст. откл-е [м] | Динамический уровень жидкости в скважине ст. откл-е | Вещественный |
| **46** | Mн [сПз] ТР | ? | Вещественный |
| **47** | Pбуф медиана [атм] | Буферное давление медиана | Вещественный |
| **48** | Pбуф среднее [атм] | Буферное давление среднее | Вещественный |
| **49** | Pбуф ст. откл-е [атм] | Буферное давление ст. откл-е | Вещественный |
| **50** | Pлин медиана [атм] | Давление после штуцера медиана | Вещественный |
| **51** | Pлин среднее [атм] | Давление после штуцера среднее  | Вещественный |
| **52** | Pлин ст. откл-е [атм] | Давление после штуцера ст. откл-е | Вещественный  |
| **53** | Pнас-я [ат] ТР | Давление насыщения | Вещественный |
| **54** | Tпл [град Ц] ТР |Пластовая температура | Вещественный |
| **55** | pв [г/см3] ТР | Давление вытеснения | Вещественный |
| **56** | Каталог ЭЦН. Производитель | Уникальный номер производителя по перчню ЭЦН | Категориальный |
| **57** | Диаметр ЭЦН [мм] | Значение диаметра ЭЦН | Вещественный |
| **58** | Fитог среднее [Гц] | Итоговая частота вращения среднее | Вещественный |
| **59** | Fитог ст. откл-е [Гц] | Итоговая частота вращения ст. откл-е | Вещественный |
| **60** | Fитог медиана [Гц] | Итоговая частота вращения медиана | Вещественный |
| **61** | pн [г/см3] ТР | Кислотность | Вещественный |
| **62** | Гф [м3/тн] МЭР |Газовый фактор | Вещественный |
| **63** | Минуты вращения насоса | Число минут вращения настса | Целочисленный |
| **64** | Сутки вращения насоса | Число суток вращения насоса | Целочисленный |
| **65** | Типоразмер ЭЦН | Габариты электроприводного центробежного насоса | Категориальный |
| **66** | Энергоэффективность ЭЦН |Энергоэффективность электроприводного центробежного насоса | Категориальный |
| **67** | dL динамика [м] ТР | Значение удлинения ствола динамика в метрах | Вещественный |
| **68** | L насос [м] ТР | Значение длины нососа в матрах | Вещественный |
| **69** | dL насос [м] ТР | Значение удлинения ствола насоса в метрах | Вещественный |
| **70** | H насос [м] ТР | Значение напора насоса | Вещественный |
| **71** | L вдп [м] ТР | Значение длина системы волоподачи в метрах | Вещественный |
| **72** | dL вдп [м] ТР | Значение удлинения ствола системы водоподачи в метрах | Вещественный |
| **73** | H вдп [м] ТР | Значение напора системы водоподачи | Вещественный |
| **74** | dH вдп-насос [м] ТР | Значение расширения напора насоса системы водоподачи | Вещественный |
| **75** | dL вдп-насос ТР | Значение удлинения ствола насоса системы водоподачи | Вещественный |
| **76** | D колонны внутренний [мм] ТР | Диаметр внутренней колонны в милиметрах | Вещественный |
| **77** | D нкт внутренний [мм] ТР | Диаметр внутренней насосно-компрессорной трубы в милиметрах | Вещественный |
| **78** | L нкт [м] НКТ | Длина насосно-компрессорной трубы в милиметрах | Вещественный |
| **79** | D нкт внешний min [мм] НКТ | Минимальный диаметр внешней насосно-компрессорной трубы в милиметрах | Вещественный |
| **80** | D нкт внутренний min [мм] НКТ | Минимальный диаметр внутренней насосно-компрессорной трубы в милиметрах | Вещественный |
| **81** | Qж мгн медиана [м3/сут] | Мгновенный медианный дебит нефти | Вещественный |
| **82** | Qж мгн среднее [м3/сут] | Мгновенный средний дебит нефти | Вещественный |
| **83** | Qж мгн ст. откл-е [м3/сут] | Мгновенное среднеквадратическое отклонение дебита нефти | Вещественный |
| **84** | К подачи на устье по среднему | Среднее количество вещества, подаваемого на устье установки | Вещественный |
| **85** | Эра.Дата демонтажа | Дата (ДД.ММ.ГГГГ), когда был произведен демонтаж насоса | Агрегат данных (дата) |
| **86** | Дата первой работы сборки | Дата (ДД.ММ.ГГГГ), когда было начало работы сборки | Агрегат данных (дата) |
| **87** | Дней между монтажом и демонтажем | Сколько дней прошло между датой монтажа и демонтажа  | Целочисленный |   
| **88** | Загрузка ПЭД медиана [%] | отношение рабочего тока к номинальному медиана | Вещественный |
| **89** | Загрузка ПЭД среднее [%] | отношение рабочего тока к номинальному среднее | Вещественный |
| **90** | Загрузка ПЭД ст. откл-е [%] | отношение рабочего тока к номинальному ст. откл-е| Вещественный |
| **91** | Ток фазы A медиана [%] | процент тока в фазе А(?) медиана| Вещественный |
| **92** | Ток фазы A среднее [%] | процент тока в фазе А(?) среднее| Вещественный |
| **93** | Ток фазы A ст. откл-е [%] | процент тока в фазе А(?) ст. откл-е | Вещественный |
| **94** | key_well_id | Сцепка идентификатора скважины и даты | Категориальный |

Целевым значением выступают 2 признака:
1. "Дней в работе" (число дней, которое оборудование с данными параметрами проработало до выхода из исследования);
2. "Признак отказа" (бинарная метка выхода из исследования: если значение равняется 1, значит, оборудование с данными характеристиками действительно вышло из строя в момент времени "Дней в работе", иначе -- оборудование к указанному дню либо продолжило работать, либо наблюдение за ним прекратилось по тем или иным причинам).

## 1.1. <a id='toc1_1_'></a>[Research](#toc0_)

### 1.1.1. <a id='toc1_1_1_'></a>[Загрузка исходных данных](#toc0_)

Благодаря сформированному виртуальному окружению можно приступить к загрузке данных.

Данные представлены в формате .csv: это значит, что нам необходимо воспользоваться средствами библиотеки pandas, чтобы прочитать, распарсить и проанализировать полученный объект.
Воспользуемся методом ```pd.read_csv()``` для того, чтобы прочитать файл с данными и представить его в памяти.

Данный метод принимает на вход:
1. путь до файла, который необходимо прочитать;
2. разделитель (символ, который отделяет разные записи друг от друга);
3. кодировка (кодировка символов в файле);
4. и т.д.
Прочитаем файл при помощи метода и результат положим в переменную df, с которой будем в дальнейшем работать.

В данном случае параметры разделителя и кодировки передавать нет необходимости, так как для данного файла они совпадают с параметрами по умолчанию: запятая и UTF-8 соответственно.
Метод вернет объект датафрейма (абстракция над табличными данными и табличными представлениями, предоставляющая возможность реализовывать логику по работе с плоскими данными), который необходимо отобразить после прочтения файла.
Также в качестве параметров укажем наименования признаков, которые необходимо представить в формате даты. Для этого в параметр parse_dates передадим список наименований колонок с датами, а также поставим флаг ```dayfirst=True``` для корректного представления.
Сам датасет представляет из себя набор объектов, каждый из которых является комбинацией технологических параметров работы добывающего насосного оборудования, а также параметров работы данного оборудования. Данные параметры составляют матрицу объект-признак.

В переменную `df_dead` поместим информацию об объектах, которые были демонтированы. Если у объекта из данного датасета признак отказа равен 0, значит, этот объект был демонтирован по причине, отличающейся от выхода из строя (такие объекты нас не интересуют, потому они помечаются нулем).

В переменную `df_live` поместим информацию об объектах, которые продолжают работу (не были демонтированы). Если у объекта из данного датасета признак отказа равен 1, значит, этот объект вышел из строя, однако не был демонтирован (такие объекты нас интересуют, потому они помечаются единицей).

In [6]:
df_dead = pd.read_excel('./output_ds_3_dead.xlsx', parse_dates=[
    'ЭРА.Дата монтажа', 'ЭРА.Дата демонтажа', 'Дата первой работы сборки', 'Дата начала экспл-ии'])
df_live = pd.read_excel('./output_ds_3_live.xlsx', parse_dates=[
    'ЭРА.Дата монтажа', 'ЭРА.Дата демонтажа', 'Дата первой работы сборки', 'Дата начала экспл-ии'])

Поскольку в датасетах данные однородны, их можно объединить, чтобы работать с данными как с одним целым.

In [ ]:
df = pd.concat((df_dead, df_live), ignore_index=True)
df

### 1.1.2. <a id='toc1_1_2_'></a>[Анализ типов данных и значений признаков](#toc0_)

Первоначально необходимо проанализировать типы данных колонок датафрейма, чтобы правильно представить исходные данные.

Поскольку для дальнейшего моделирования необходимо, чтобы все значения признаков являлись вещественными числами (либо специальным значением NaN, обозначающим нечисловое значение), воспользуемся методом датафрейма ```.info()```, который выводит сводную информацию о колонках объекта. После этого выполним преобразование категориальных данных и данных с пропусками, чтобы получить готовый набор данных для работы.

#### 1.1.2.1. <a id='toc1_1_2_1_'></a>[Выделение фрагментов исходного датасета](#toc0_)

Исходный датасет имеет большое количество признаков: метод ```DataFrame.info()``` не сможет корректно отобразить сводную информацию о типах данных признаков. Поэтому выделим $n = 8$ участков, которые будут содержать участки признакового пространства, представленные в корректном виде для дальнейшего анализа.

In [8]:
import math


N = len(df.columns)
n = 8
h = math.ceil(N / n)
intervals = [(i * h, (i + 1) * h) for i in range(n)]
intervals

[(0, 12), (12, 24), (24, 36), (36, 48), (48, 60), (60, 72), (72, 84), (84, 96)]

В переменную `df_parts` поместим данные фрагменты датасета в соответствии с интервалами разбиения.

In [9]:
columns = df.columns
df_parts = [
    df[columns[interval[0]: interval[1]]] for interval in intervals
]

#### 1.1.2.2. <a id='toc1_1_2_2_'></a>[Анализ выделенных фрагментов](#toc0_)

Рассмотрим первый фрагмент датасета. Первоначально выведем его на экран.

In [ ]:
df_parts[0].head()

In [11]:
df_parts[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7082 entries, 0 to 7081
Data columns (total 12 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   key_well_sborka                   7082 non-null   object        
 1   well_id                           7082 non-null   int64         
 2   sborka_id                         7082 non-null   int64         
 3   Скважина                          7082 non-null   object        
 4   Сборка                            7081 non-null   object        
 5   Причины остановок новые           7082 non-null   object        
 6   Признак отказа                    7082 non-null   int64         
 7   ЭРА.Дата монтажа                  7082 non-null   datetime64[ns]
 8   ЭРА.Дата демонтажа                7082 non-null   datetime64[ns]
 9   Дней между монтажом и демонтажем  4352 non-null   float64       
 10  Дата первой работы сборки         7082 non-null 

Рассмотрим некоторый набор признаков на содержание уникальных значений (отобранный по результатам изучения выведенного на экран фрагмента), чтобы принять решение: удалять данные признаки или же включать в набор категориальных.

In [12]:
df_parts[0].nunique()

key_well_sborka                     7082
well_id                             3193
sborka_id                           7082
Скважина                            3193
Сборка                               734
Причины остановок новые             3703
Признак отказа                         2
ЭРА.Дата монтажа                    1858
ЭРА.Дата демонтажа                  1609
Дней между монтажом и демонтажем    1294
Дата первой работы сборки           1879
Дней в работе                       1488
dtype: int64

Видим, что признаки 'key_well_sborka', 'well_id', 'sborka_id', 'Скважина', 'Сборка' и 'Причины остановок новые' имеют большую концентрацию уникальных значений, значит, их нельзя считать категориальными признаками: они подлежат удалению из исходного набора данных, поскольку это идентификаторы записей в базе данных.

`Исправление после валидации` Также признак 'Причины остановок новые' является текстовым: оператор при остановке оборудования пишет служебные сообщения, они становятся известными только на момент выхода объекта из строя, потому его необходимо исключить.

Также признак 'ЭРА.Дата демонтажа' известен только на тот момент, когда оборудование вышло из строя. Получаем, что его нельзя использовать при построении моделей, так как это вызовет утечку данных. Аналогичная ситуация с признаком 'Дней между монтажом и демонтажем'.

In [13]:
df.drop(columns=['key_well_sborka', 'well_id', 'sborka_id', 'Скважина',
        'Сборка', 'Причины остановок новые', 'ЭРА.Дата демонтажа', 'Дней между монтажом и демонтажем'], inplace=True)

Также заменим агрегат данных 'ЭРА.Дата монтажа' на три производных признака: день даты монтажа, а также ее месяц и год. Исходный признак удалим. Аналогичную работу проделаем с агрегатом 'Дата первой работы сборки'.

In [15]:
col = 'ЭРА.Дата монтажа'

df[f'{col}_day'] = df[col].dt.day
df[f'{col}_month'] = df[col].dt.month
df[f'{col}_year'] = df[col].dt.year
df.drop(columns=[col], inplace=True)

In [16]:
col = 'Дата первой работы сборки'

df[f'{col}_day'] = df[col].dt.day
df[f'{col}_month'] = df[col].dt.month
df[f'{col}_year'] = df[col].dt.year
df.drop(columns=[col], inplace=True)

-----------------

Анализ и очистку данного фрагмента проведем аналогично первому фрагменту.

In [ ]:
df_parts[1].head()

In [18]:
df_parts[1].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7082 entries, 0 to 7081
Data columns (total 12 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Минуты вращения насоса           7082 non-null   float64
 1   Сутки вращения насоса            7082 non-null   float64
 2   Кол-во циклов ПКВ                7082 non-null   float64
 3   Кол-во циклов ПКВ/Дней в работе  7082 non-null   float64
 4   Режим гр-ка                      7082 non-null   object 
 5   Группа                           7082 non-null   object 
 6   Группа ПКВ                       7082 non-null   object 
 7   УРЭ [кВтч/т]                     6939 non-null   float64
 8   УРЭ К [кВтч/т]                   6945 non-null   float64
 9   Э/э (Wакт) [кВт*ч]               7082 non-null   float64
 10  Э/э реакт. (ТМ)                  7082 non-null   float64
 11  Э/э счет. (ТМ) [Вт]              7082 non-null   float64
dtypes: float64(9), objec

In [19]:
df_parts[1].nunique()

Минуты вращения насоса             7079
Сутки вращения насоса              7079
Кол-во циклов ПКВ                  6763
Кол-во циклов ПКВ/Дней в работе    6810
Режим гр-ка                          20
Группа                                4
Группа ПКВ                            4
УРЭ [кВтч/т]                       6939
УРЭ К [кВтч/т]                     6944
Э/э (Wакт) [кВт*ч]                 6946
Э/э реакт. (ТМ)                    6117
Э/э счет. (ТМ) [Вт]                6936
dtype: int64

Удалим признаки, которые вычисляются на основании других, чтобы предотвратить утечку данных.

In [20]:
df[['Минуты вращения насоса', 'Сутки вращения насоса']].corr()

,Минуты вращения насоса,Сутки вращения насоса
Минуты вращения насоса,1.0,1.0
Сутки вращения насоса,1.0,1.0


In [21]:
df.drop(columns=['Минуты вращения насоса', 'Кол-во циклов ПКВ/Дней в работе'], inplace=True)

----------------------------------

Анализ и очистку данного фрагмента проведем аналогично первому фрагменту.

In [ ]:
df_parts[2].head()

In [23]:
df_parts[2].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7082 entries, 0 to 7081
Data columns (total 12 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Qж OIS протяжка среднее [м3/сут]      7062 non-null   float64
 1   Qж OIS протяжка медиана [м3/сут]      7062 non-null   float64
 2   Qж OIS протяжка ст. откл-е [м3/сут]   7058 non-null   float64
 3   Qж мгн среднее [м3/сут]               7062 non-null   float64
 4   Qж мгн медиана [м3/сут]               7062 non-null   float64
 5   Qж мгн ст. откл-е [м3/сут]            7058 non-null   float64
 6   Qн OIS протяжка среднее [тн/сут]      7062 non-null   float64
 7   Qн OIS протяжка медиана [тн/сут]      7062 non-null   float64
 8   Qн OIS протяжка ст. откл-е [тн/сут]   7058 non-null   float64
 9   Обв-ть V OIS протяжка среднее [%]     7062 non-null   float64
 10  Обв-ть V OIS протяжка медиана [%]     7062 non-null   float64
 11  Обв-ть V OIS прот

-----------------

Анализ и очистку данного фрагмента проведем аналогично первому фрагменту.

In [ ]:
df_parts[3].head()

In [25]:
df_parts[3].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7082 entries, 0 to 7081
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Pзаб среднее [ат]       7070 non-null   float64
 1   Pзаб медиана [ат]       7070 non-null   float64
 2   Pзаб ст. откл-е [ат]    7067 non-null   float64
 3   Pпласт среднее [ат]     6833 non-null   float64
 4   Pпл медиана [ат]        6833 non-null   float64
 5   Pпласт ст. откл-е [ат]  6833 non-null   float64
 6   Кол-во пластов          7082 non-null   int64  
 7   Глубина насоса [м]      7082 non-null   float64
 8   Fитог среднее [Гц]      7082 non-null   float64
 9   Fитог медиана [Гц]      7082 non-null   float64
 10  Fитог ст. откл-е [Гц]   7073 non-null   float64
 11  Pбуф среднее [атм]      7056 non-null   float64
dtypes: float64(11), int64(1)
memory usage: 664.1 KB


-----------------------------

Анализ и очистку данного фрагмента проведем аналогично первому фрагменту.

In [26]:
df_parts[4].head()

,Pбуф медиана [атм],Pбуф ст. откл-е [атм],Pлин среднее [атм],Pлин медиана [атм],Pлин ст. откл-е [атм],Hдин среднее [м],Hдин медиана [м],Hдин ст. откл-е [м],Загрузка ПЭД среднее [%],Загрузка ПЭД медиана [%],Загрузка ПЭД ст. откл-е [%],Ток фазы A среднее [А]
0,28.3,0.704870,29.197531,29.25,2.133959,1083.563380,1079.0,270.109699,23.981481,23.5,8.043492,9.725909
1,18.3,1.366678,14.000972,16.70,6.771844,2352.835650,2351.0,40.397807,52.293883,56.0,9.681344,9.000277
2,15.4,0.659779,14.370403,14.40,0.645894,2369.826733,2401.0,154.580485,70.222222,69.0,11.220006,18.473596
3,14.2,0.384983,13.223622,13.20,0.498076,2355.052424,2368.0,93.347478,62.996169,61.0,7.875226,14.267363
4,13.9,0.301033,13.000000,13.00,0.202312,2093.937500,2045.0,96.092023,69.512821,70.0,21.263846,13.649474


In [27]:
df_parts[4].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7082 entries, 0 to 7081
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Pбуф медиана [атм]           7056 non-null   float64
 1   Pбуф ст. откл-е [атм]        7055 non-null   float64
 2   Pлин среднее [атм]           6962 non-null   float64
 3   Pлин медиана [атм]           6962 non-null   float64
 4   Pлин ст. откл-е [атм]        6955 non-null   float64
 5   Hдин среднее [м]             7056 non-null   float64
 6   Hдин медиана [м]             7056 non-null   float64
 7   Hдин ст. откл-е [м]          7055 non-null   float64
 8   Загрузка ПЭД среднее [%]     6966 non-null   float64
 9   Загрузка ПЭД медиана [%]     6966 non-null   float64
 10  Загрузка ПЭД ст. откл-е [%]  6963 non-null   float64
 11  Ток фазы A среднее [А]       6967 non-null   float64
dtypes: float64(12)
memory usage: 664.1 KB


------------

Анализ и очистку данного фрагмента проведем аналогично первому фрагменту.

In [ ]:
df_parts[5].head()

In [29]:
df_parts[5].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7082 entries, 0 to 7081
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Ток фазы A медиана [А]         6967 non-null   float64
 1   Ток фазы A ст. откл-е [А]      6965 non-null   float64
 2   Гф [м3/тн] МЭР                 7070 non-null   float64
 3   Pнас-я [ат] ТР                 7082 non-null   float64
 4   Bн [м3/м3] ТР                  7082 non-null   float64
 5   Tпл [град Ц] ТР                7082 non-null   float64
 6   Mн [сПз] ТР                    7082 non-null   float64
 7   pн [г/см3] ТР                  7082 non-null   float64
 8   pв [г/см3] ТР                  7082 non-null   float64
 9   К подачи на устье по среднему  6995 non-null   float64
 10  Диаметр ЭЦН [мм]               7080 non-null   float64
 11  Каталог ЭЦН. Производитель     7080 non-null   object 
dtypes: float64(11), object(1)
memory usage: 664.1+ K

In [30]:
df['Каталог ЭЦН. Производитель'].nunique()

12

In [31]:
df['Каталог ЭЦН. Производитель'].unique()

array(['Новомет', 'Schlumberger', 'Борец', 'Новые технологии', 'Лемаз',
       'Алнас', 'СЦ ЭПУ', 'Reda', 'EZLine', 'Алмаз',
       'Borets International', 'Ижнефтепласт', nan], dtype=object)

---------

Анализ и очистку данного фрагмента проведем аналогично первому фрагменту.

In [ ]:
df_parts[6].head()

In [33]:
df_parts[6].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7082 entries, 0 to 7081
Data columns (total 12 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Номинальный дебит сборки [м3/сут]  7069 non-null   float64
 1   Номинальный напор сборки [м]       6989 non-null   float64
 2   Номинальная мощность сборки [кВт]  7068 non-null   float64
 3   Типоразмер ЭЦН                     7018 non-null   object 
 4   Энергоэффективность ЭЦН            7080 non-null   object 
 5   L вдп [м] ТР                       7082 non-null   float64
 6   dL вдп [м] ТР                      7075 non-null   float64
 7   H вдп [м] ТР                       7075 non-null   float64
 8   L насос [м] ТР                     7082 non-null   float64
 9   dL насос [м] ТР                    7075 non-null   float64
 10  H насос [м] ТР                     7075 non-null   float64
 11  dH вдп-насос [м] ТР                7074 non-null   float

In [34]:
df[['Типоразмер ЭЦН', 'Энергоэффективность ЭЦН']].nunique()

Типоразмер ЭЦН             5
Энергоэффективность ЭЦН    4
dtype: int64

----------

Анализ и очистку данного фрагмента проведем аналогично первому фрагменту.

In [ ]:
df_parts[7].head()

In [36]:
df_parts[7].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7082 entries, 0 to 7081
Data columns (total 7 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   dL вдп-насос ТР                7082 non-null   float64       
 1   D колонны внутренний [мм] ТР   7072 non-null   float64       
 2   D нкт внешний min [мм] НКТ     7076 non-null   float64       
 3   D нкт внутренний min [мм] НКТ  7076 non-null   float64       
 4   Lнкт [м] НКТ                   7076 non-null   float64       
 5   Тип ствола                     7082 non-null   object        
 6   Дата начала экспл-ии           7082 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 387.4+ KB


In [37]:
df['Тип ствола'].nunique()

3

In [38]:
col = 'Дата начала экспл-ии'

df[f'{col}_day'] = df[col].dt.day
df[f'{col}_month'] = df[col].dt.month
df[f'{col}_year'] = df[col].dt.year
df.drop(columns=[col], inplace=True)

------------

Также выполним приведение типов: категориальным признакам назначим тип 'str', чтобы корректно выполнялись операции в библиотеке lifelines (параметр strata).

In [39]:
object_columns = df.select_dtypes('object').columns
df[object_columns] = df[object_columns].astype(str)

И подсчитаем количество вещественных признаков.

In [40]:
_, n_numeric_features = df.select_dtypes('number').shape
n_numeric_features

80

### 1.1.3. <a id='toc1_1_3_'></a>[Формирование обучающей и тестирующей выборок](#toc0_)

Для построения статистических моделей, а также их проверки, необходимо сформировать обучающий и тестовый наборы данных.

При помощи метода `train_test_split` сформируем обучающую и тестовую выборки. Поскольку в исходном датасете около 7000 записей, на тест оставим 15% данных. При этом обязательно выполним стратификацию, чтобы и в train-, и в test-выборках сохранялась пропорция цензурированных данных.

Дополнительно, перемешаем данные, чтобы в них не было временных и порядковых зависимостей, которые могли возникнуть при сборе датаета и порядке хранения записей в БД.

Необходимо выделить обучающий и тестовый датасеты именно на данном этапе, чтобы не допустить утечки данных.

In [51]:
from sklearn.model_selection import train_test_split

`Исправление после валидации` На тест было отправлено не по одному экземпляру каждых данных (с разным признаком отказа), а была сформирована целая выборка + устранена проблема с неправильной интерпретацией признака отказа (когда 0 и 1 были поменяны местами). 

In [52]:
y_columns = ['Дней в работе', 'Признак отказа']

train_df, test_df = train_test_split(df, shuffle=True, stratify=df['Признак отказа'], random_state=seed, test_size=0.15)

X_train, X_test = train_df.drop(columns=y_columns), test_df.drop(columns=y_columns)
y_train, y_test = train_df[y_columns], test_df[y_columns]